In [1]:
#!pip install bezier
#!pip install networkx==2.6.2

import copy
import getopt
import math
import networkx as nx
import operator
import os
import random
import sys
from scipy import stats
import seaborn as sns
import time
import pandas as pd
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.misc import derivative
from matplotlib.collections import LineCollection
from scipy.stats import pearsonr
from networkx.algorithms import tree
try: 
    import bezier
except:
    pass
%matplotlib inline 

In [2]:
palette = sns.color_palette("flare")
palette

[(0.9155979, 0.55210684, 0.42070204),
 (0.888292, 0.40830288, 0.36223756),
 (0.81942908, 0.28911553, 0.38102921),
 (0.69226314, 0.23413578, 0.42480327),
 (0.56041794, 0.19845221, 0.44207535),
 (0.42355299, 0.16934709, 0.42581586)]

In [3]:
try: 
    from google.colab import drive
    drive.mount('/content/gdrive')
except:
    pass

In [4]:
def clean(series):
    series = series.str.upper()
    series = series.str.replace(u"Á", "A")
    series = series.str.replace(u"É", "E")
    series = series.str.replace(u"Ú", "U")
    series = series.str.replace(u"Ó", "O")
    series = series.str.replace(u"Í", "I")
    series = series.str.replace(u"Ã", "A")
    series = series.str.replace(u"Õ", "O")
    series = series.str.replace(u"Ô", "O")
    series = series.str.replace(u"Â", "A")
    series = series.str.replace(u"Ê", "E")
    series = series.str.replace(u"Ç", "C")
    series = series.str.replace(u"-", "")
    series = series.str.replace(u"'", "")
    series = series.str.upper()
    return series

def clean_str(series):
    series = series.upper()
    series = series.replace(u"Á", "A")
    series = series.replace(u"É", "E")
    series = series.replace(u"Ú", "U")
    series = series.replace(u"Ó", "O")
    series = series.replace(u"Í", "I")
    series = series.replace(u"Ã", "A")
    series = series.replace(u"Õ", "O")
    series = series.replace(u"Ô", "O")
    series = series.replace(u"Â", "A")
    series = series.replace(u"Ê", "E")
    series = series.replace(u"Ç", "C")
    series = series.replace(u"-", "")
    series = series.replace(u"'", "")
    return series

In [5]:
try:
    popDf = pd.read_excel("/content/gdrive/MyDrive/Graph Theory for Machine Learning/gcavali-repo-main/MUNIC_2019.xlsx")
except:
    popDf = pd.read_excel("MUNIC_2019.xlsx")
popDf['NOME MUNIC'] = clean(popDf['NOME MUNIC'])
popDf.head(2)

,CodMun,REGIAO,COD UF,UF,NOME MUNIC,POP EST,CLASSE POP
0,1100015,1 - Norte,11,RO,ALTA FLORESTA DOESTE (RO),22945,4 - 20001 até 50000
1,1100023,1 - Norte,11,RO,ARIQUEMES (RO),107863,6 - 100001 até 500000


In [6]:
try:
    casosDf = pd.read_csv('/content/gdrive/MyDrive/Graph Theory for Machine Learning/gcavali-repo-main/caso_full.csv')
except:
    casosDf = pd.read_csv('caso_full.csv')
    
casosStDf = casosDf[casosDf.place_type == 'state']
casosDf = casosDf[casosDf.place_type == 'city']
casosDf.city = clean(casosDf.city)
casosDf['city_state'] = casosDf.city+' ('+casosDf.state+')'
casosDf.head(2) # casos por cidade

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths,city_state
0,RIO BRANCO,1200401.0,2020-03-17,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-17,0.0,0,1,city,AC,3,0,RIO BRANCO (AC)
2,RIO BRANCO,1200401.0,2020-03-18,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-18,0.0,0,2,city,AC,0,0,RIO BRANCO (AC)


In [7]:
casosStDf.head(2) # casos por estado

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
1,NaN,12.0,2020-03-17,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0,0,1,state,AC,3,0
3,NaN,12.0,2020-03-18,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0,0,2,state,AC,0,0


In [8]:
epiweekDf = casosDf[['date','epidemiological_week']].drop_duplicates()
print(len(epiweekDf),"semanas epidemiológicas")
epiweekDf.tail(2) # semanas epidemiológicas

692 semanas epidemiológicas


,date,epidemiological_week
2970939,2020-03-03,202010
2970941,2020-03-04,202010


In [9]:
casosDf = pd.merge(left=casosDf, right=popDf, how='left', left_on='city_state', right_on='NOME MUNIC')
print(casosDf['CLASSE POP'].value_counts())

3 - 10001 até 20000      828311
1 - Até 5000             739884
2 - 5001 até 10000       723726
4 - 20001 até 50000      693037
5 - 50001 até 100000     225184
6 - 100001 até 500000    181618
7 - Maior que 500000      32236
Name: CLASSE POP, dtype: int64


In [10]:
def group_data(Df,list):
  GroupDf = Df.groupby(list).agg({'last_available_confirmed':'sum',
                                          'last_available_confirmed_per_100k_inhabitants':'mean', 
                                          'new_confirmed':'sum',
                                          'last_available_deaths':'sum', 
                                          'new_deaths':'sum', 
                                          'estimated_population':'mean'}).reset_index()
  return GroupDf

def create_variable(Df, level, x1, x2):
    Df['confirmed_percent'] = Df.last_available_confirmed/Df.estimated_population
    Df['new_deaths_percent'] = Df.groupby(level)['new_deaths'].pct_change()
    Df['new_confirmed_percent'] = Df.groupby(level)['new_confirmed'].pct_change()
    Df['new_deaths_percent'] = Df['new_deaths_percent'].replace([np.inf, -np.inf, np.nan, None], None)
    Df['new_confirmed_percent'] = Df['new_confirmed_percent'].replace([np.inf, -np.inf, np.nan, None], None)
    Df['decrease_deaths'] = Df['new_deaths_percent']<= x1
    Df['stability_deaths'] = (Df['new_deaths_percent']> x1) & (Df['new_deaths_percent'] <= x2)
    Df['increase_deaths'] = Df['new_deaths_percent'] > x2
    Df['decrease_cases'] = Df['new_confirmed_percent']< x1
    Df['stability_cases'] = (Df['new_confirmed_percent']> x1) & (Df['new_confirmed_percent']<= x2)
    Df['increase_cases'] = Df['new_confirmed_percent']> x2
    Df['confirmed_t1'] = [None] + list(Df['last_available_confirmed'][:-1])
    Df['confirmed_t2'] = [None, None] + list(Df['last_available_confirmed'][:-2])
    Df['new_confirmed_t1'] = [None] + list(Df['new_confirmed'][:-1])
    Df['new_confirmed_t2'] = [None, None] + list(Df['new_confirmed'][:-2])
    Df['new_confirmed_t3'] = [None, None, None] + list(Df['new_confirmed'][:-3])
    Df['new_confirmed_t4'] = [None, None, None, None] + list(Df['new_confirmed'][:-4])
    Df['new_confirmed_t5'] = [None, None, None, None, None] + list(Df['new_confirmed'][:-5])
    Df['new_confirmed_t6'] = [None, None, None, None, None, None] + list(Df['new_confirmed'][:-6])
    return Df

def group_time(Df, level_time, level_city):
  GroupDf = Df.groupby(level_time).agg({level_city:'count',
                                  'decrease_deaths': 'sum',
                                  'stability_deaths': 'sum',
                                  'increase_deaths': 'sum',
                                  'decrease_cases': 'sum',
                                  'stability_cases': 'sum',
                                  'increase_cases': 'sum'
                                 })
  GroupDf['decrease_deaths_percent'] = GroupDf['decrease_deaths']/GroupDf[level_city]
  GroupDf['stability_deaths_percent'] = GroupDf['stability_deaths']/GroupDf[level_city]
  GroupDf['increase_deaths_percent'] = GroupDf['increase_deaths']/GroupDf[level_city]
  GroupDf['decrease_cases_percent'] = GroupDf['decrease_cases']/GroupDf[level_city]
  GroupDf['stability_cases_percent'] = GroupDf['stability_cases']/GroupDf[level_city]
  GroupDf['increase_cases_percent'] = GroupDf['increase_cases']/GroupDf[level_city]
  return GroupDf

In [11]:
casosStGroupDf = group_data(casosStDf,['state','epidemiological_week'])
casosStGroupDf = create_variable(casosStGroupDf, 'state', -0.01, 0.10)
casosStGroupDf.head(2)

,state,epidemiological_week,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,estimated_population,confirmed_percent,new_deaths_percent,...,stability_cases,increase_cases,confirmed_t1,confirmed_t2,new_confirmed_t1,new_confirmed_t2,new_confirmed_t3,new_confirmed_t4,new_confirmed_t5,new_confirmed_t6
0,AC,202012,28,0.626068,11,0,0,894470.0,0.000031,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AC,202013,145,2.315819,14,0,0,894470.0,0.000162,NaN,...,False,True,28.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN


In [12]:
casosGroupDf = group_data(casosDf,['city_state','epidemiological_week'])
casosGroupDf = create_variable(casosGroupDf, 'city_state', -0.01, 0.10)
casosGroupDf.head(2)

,city_state,epidemiological_week,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,estimated_population,confirmed_percent,new_deaths_percent,...,stability_cases,increase_cases,confirmed_t1,confirmed_t2,new_confirmed_t1,new_confirmed_t2,new_confirmed_t3,new_confirmed_t4,new_confirmed_t5,new_confirmed_t6
0,ABADIA DE GOIAS (GO),202021,16,59.537100,5,0,0,8958.0,0.001786,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABADIA DE GOIAS (GO),202022,41,65.384496,1,0,0,8958.0,0.004577,NaN,...,False,False,16.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN


In [13]:
casosGroupDf.groupby('epidemiological_week')['new_confirmed'].sum().to_csv("cases_week.csv")

In [14]:
casosEpiWeekDf = group_time(casosGroupDf, 'epidemiological_week','city_state')
casosEpiWeekDf.head()

,city_state,decrease_deaths,stability_deaths,increase_deaths,decrease_cases,stability_cases,increase_cases,decrease_deaths_percent,stability_deaths_percent,increase_deaths_percent,decrease_cases_percent,stability_cases_percent,increase_cases_percent
epidemiological_week,,,,,,,,,,,,,
202009,1,1,0,0,1,0,0,1.000000,0.0,0.000000,1.000000,0.000000,0.000000
202010,6,6,0,0,5,0,1,1.000000,0.0,0.000000,0.833333,0.000000,0.166667
202011,31,31,0,0,27,1,3,1.000000,0.0,0.000000,0.870968,0.032258,0.096774
202012,142,142,0,0,118,5,19,1.000000,0.0,0.000000,0.830986,0.035211,0.133803
202013,313,312,0,1,230,18,65,0.996805,0.0,0.003195,0.734824,0.057508,0.207668


In [15]:
casosStEpiWeekDf = group_time(casosStGroupDf, 'epidemiological_week','state')
casosStEpiWeekDf.head()

,state,decrease_deaths,stability_deaths,increase_deaths,decrease_cases,stability_cases,increase_cases,decrease_deaths_percent,stability_deaths_percent,increase_deaths_percent,decrease_cases_percent,stability_cases_percent,increase_cases_percent
epidemiological_week,,,,,,,,,,,,,
202009,1,1,0,0,1,0,0,1.000000,0.0,0.000000,1.000000,0.000000,0.000000
202010,5,5,0,0,4,0,1,1.000000,0.0,0.000000,0.800000,0.000000,0.200000
202011,17,17,0,0,12,0,5,1.000000,0.0,0.000000,0.705882,0.000000,0.294118
202012,27,26,0,0,10,0,16,0.962963,0.0,0.000000,0.370370,0.000000,0.592593
202013,27,24,0,2,1,2,24,0.888889,0.0,0.074074,0.037037,0.074074,0.888889


In [16]:
def create_network(path):
    """
    Create a NetworkX graph object using the airport and route databases.
    Args:
        nodes: The file path to the nodes .csv file.
        edeges: The file path to the edges .csv file.
    Returns:
        G: A NetworkX DiGraph object populated with the nodes and edges assigned
           by the data files from the arguments.
    """
    print("Creating network.")
    G = nx.read_gexf(path)
    
    # Add pos attribute in nodes
    for node in G.nodes():
        long = G.nodes[node]['long']
        lat = G.nodes[node]['lat']
        loc = (long, lat)
        node_dict = {node : loc}
        nx.set_node_attributes(G, node_dict, 'pos')
    
    # Calculate the edge weights
    print("\tCalculating edge weights",end="")
    degree_network = nx.Graph(G)
    ldegree = degree_network.degree
    for i,j in G.edges():
        degree_sum = ldegree[i] + ldegree[j]
        G[i][j]['weight'] = degree_sum
    
    print("\t\t\t\t[Done]")
    
    G = nx.relabel_nodes(G, clean_str)
    
    # Calculate the edge distances
    print("\tCalculating edge distance",end="")
    G = calculate_distance(G)
    print("\t\t\t\t[Done]")

    # Add clustering data
    print("\tCalculating clustering coefficents",end="")
    cluster_network = nx.Graph(G)
    lcluster = nx.clustering(cluster_network)
    for i,j in G.edges():
        cluster_sum = lcluster[i] + lcluster[j]
        G[i][j]['cluster'] = cluster_sum
    print("\t\t\t[Done]")
    
    return G

In [17]:
from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a))

def calculate_distance(input_network):
    """
    Add weights to the edges of a network based on the degrees of the connecting
    verticies, and return the network.
    Args:
        input_network: A NetworkX graph object
    Returns:
        G: A weighted NetworkX graph object.
    """
    
    G = input_network.copy()

    # Add weights to edges
    for node, successor in G.edges():
        dist = distance(G.nodes[node]['lat'], G.nodes[node]['long'],
                        G.nodes[successor]['lat'], G.nodes[successor]['long'])
        edge_dict = {(node,successor): dist}
        nx.set_edge_attributes(G, edge_dict, 'distance')
        
    return G

In [18]:
def calculate_weights(input_network):
    """
    Add weights to the edges of a network based on the degrees of the connecting
    verticies, and return the network.
    Args:
        input_network: A NetworkX graph object
    Returns:
        G: A weighted NetworkX graph object.
    """
    
    G = input_network.copy()

    # Add weights to edges
    for node in G.nodes():
        successors = G.neighbors(node)
        weights = {}

        # Calculate the total out degree of all succs
        total_degree = 0
        for successor in successors: 
            weights[successor] = G.degree(successor)
            
        largest_weight = max(weights.items())[1]
        smallest_weight = min(weights.items())[1]
        
        successors = G.neighbors(node)
        for successor in successors:
            if largest_weight == smallest_weight:
                relative_weight = 0
            else: 
                relative_weight = (weights[successor] - smallest_weight) / (largest_weight - smallest_weight)    
            G[node][successor]['weight'] = relative_weight

    return G

In [19]:
def curved_edges(G, pos, dist_ratio=0.2, bezier_precision=20, polarity='random'):
    # Get nodes into np array
    edges = np.array(G.edges())
    l = edges.shape[0]

    if polarity == 'random':
        # Random polarity of curve
        rnd = np.where(np.random.randint(2, size=l)==0, -1, 1)
    else:
        # Create a fixed (hashed) polarity column in the case we use fixed polarity
        # This is useful, e.g., for animations
        rnd = np.where(np.mod(np.vectorize(hash)(edges[:,0])+np.vectorize(hash)(edges[:,1]),2)==0,-1,1)
    
    # Coordinates (x,y) of both nodes for each edge
    # e.g., https://stackoverflow.com/questions/16992713/translate-every-element-in-numpy-array-according-to-key
    # Note the np.vectorize method doesn't work for all node position dictionaries for some reason
    u, inv = np.unique(edges, return_inverse = True)
    coords = np.array([pos[x] for x in u])[inv].reshape([edges.shape[0], 2, edges.shape[1]])
    coords_node1 = coords[:,0,:]
    coords_node2 = coords[:,1,:]
    
    # Swap node1/node2 allocations to make sure the directionality works correctly
    should_swap = coords_node1[:,0] > coords_node2[:,0]
    coords_node1[should_swap], coords_node2[should_swap] = coords_node2[should_swap], coords_node1[should_swap]
    
    # Distance for control points
    dist = dist_ratio * np.sqrt(np.sum((coords_node1-coords_node2)**2, axis=1))

    # Gradients of line connecting node & perpendicular
    m1 = (coords_node2[:,1]-coords_node1[:,1])/(coords_node2[:,0]-coords_node1[:,0])
    m2 = -1/m1

    # Temporary points along the line which connects two nodes
    # e.g., https://math.stackexchange.com/questions/656500/given-a-point-slope-and-a-distance-along-that-slope-easily-find-a-second-p
    t1 = dist/np.sqrt(1+m1**2)
    v1 = np.array([np.ones(l),m1])
    coords_node1_displace = coords_node1 + (v1*t1).T
    coords_node2_displace = coords_node2 - (v1*t1).T

    # Control points, same distance but along perpendicular line
    # rnd gives the 'polarity' to determine which side of the line the curve should arc
    t2 = dist/np.sqrt(1+m2**2)
    v2 = np.array([np.ones(len(edges)),m2])
    coords_node1_ctrl = coords_node1_displace + (rnd*v2*t2).T
    coords_node2_ctrl = coords_node2_displace + (rnd*v2*t2).T

    # Combine all these four (x,y) columns into a 'node matrix'
    node_matrix = np.array([coords_node1, coords_node1_ctrl, coords_node2_ctrl, coords_node2])

    # Create the Bezier curves and store them in a list
    curveplots = []
    for i in range(l):
        nodes = node_matrix[:,i,:].T
        curveplots.append(bezier.Curve(nodes, degree=2).evaluate_multi(np.linspace(0,1,bezier_precision)).T)
      
    # Return an array of these curves
    curves = np.array(curveplots)
    
    return curves

In [20]:
try: 
  G = create_network('/content/gdrive/MyDrive/Graph Theory for Machine Learning/gcavali-repo-main/grafo_cidades_aeroportos.gexf')
except:
  G = create_network('grafo_cidades_aeroportos.gexf')
G.remove_edges_from(nx.selfloop_edges(G))

Creating network.
Creating network.
	Calculating edge weights				[Done]
	Calculating edge distance				[Done]
	Calculating clustering coefficents			[Done]


In [21]:
temp = casosGroupDf[casosGroupDf.epidemiological_week == 202040]
print(temp.columns)

Index(['city_state', 'epidemiological_week', 'last_available_confirmed',
       'last_available_confirmed_per_100k_inhabitants', 'new_confirmed',
       'last_available_deaths', 'new_deaths', 'estimated_population',
       'confirmed_percent', 'new_deaths_percent', 'new_confirmed_percent',
       'decrease_deaths', 'stability_deaths', 'increase_deaths',
       'decrease_cases', 'stability_cases', 'increase_cases', 'confirmed_t1',
       'confirmed_t2', 'new_confirmed_t1', 'new_confirmed_t2',
       'new_confirmed_t3', 'new_confirmed_t4', 'new_confirmed_t5',
       'new_confirmed_t6'],
      dtype='object')


In [22]:
epi_week = list(set(casosGroupDf.epidemiological_week.tolist()))
edges = [str(x) for x in list(G.edges())]
#edges = [str((u,v)) for u,v,e in G.edges(data=True) if e['aereo'] == 'sim']
new_confirmed_sum_df = pd.DataFrame(columns=edges, index = epi_week)
new_confirmed_diff_df = pd.DataFrame(columns=edges, index = epi_week)
new_confirmed_percent_avg_df = pd.DataFrame(columns=edges, index = epi_week)
new_confirmed_percent_diff_df = pd.DataFrame(columns=edges, index = epi_week)
new_confirmed_corr_df = pd.DataFrame(columns=edges, index = epi_week)
new_confirmed_sum_df.loc[202009, "('CACOAL (RO)', 'VARZEA GRANDE (MT)')"]

nan

In [ ]:
casosGroupDf[casosGroupDf.epidemiological_week == 202040]

In [23]:
aero_edges = [(u,v) for u,v,e in G.edges(data=True) if e['aereo'] == 'sim']
aero_edges

[('CACOAL (RO)', 'VARZEA GRANDE (MT)'),
 ('CACOAL (RO)', 'CAMPINAS (SP)'),
 ('CACOAL (RO)', 'JIPARANA (RO)'),
 ('VILHENA (RO)', 'JIPARANA (RO)'),
 ('VILHENA (RO)', 'VARZEA GRANDE (MT)'),
 ('VILHENA (RO)', 'CAMPINAS (SP)'),
 ('JUINA (MT)', 'ARIPUANA (MT)'),
 ('JUINA (MT)', 'JUARA (MT)'),
 ('JUINA (MT)', 'VARZEA GRANDE (MT)'),
 ('JUINA (MT)', 'TANGARA DA SERRA (MT)'),
 ('JUINA (MT)', 'PONTES E LACERDA (MT)'),
 ('JUINA (MT)', 'NOVA MUTUM (MT)'),
 ('JUINA (MT)', 'LUCAS DO RIO VERDE (MT)'),
 ('JIPARANA (RO)', 'CONFINS (MG)'),
 ('JIPARANA (RO)', 'VARZEA GRANDE (MT)'),
 ('JIPARANA (RO)', 'CAMPINAS (SP)'),
 ('PORTO VELHO (RO)', 'LABREA (AM)'),
 ('PORTO VELHO (RO)', 'BELEM (PA)'),
 ('PORTO VELHO (RO)', 'BRASILIA (DF)'),
 ('PORTO VELHO (RO)', 'CAMPINAS (SP)'),
 ('PORTO VELHO (RO)', 'CONFINS (MG)'),
 ('PORTO VELHO (RO)', 'FORTALEZA (CE)'),
 ('PORTO VELHO (RO)', 'GUARULHOS (SP)'),
 ('PORTO VELHO (RO)', 'MANAUS (AM)'),
 ('PORTO VELHO (RO)', 'VARZEA GRANDE (MT)'),
 ('PORTO VELHO (RO)', 'BOA VISTA (R

In [ ]:
for edge in G.edges():
    print(edge)
    city_state_0 = edge[0]
    city_state_1 = edge[1]
    temp_0 = casosGroupDf[casosGroupDf.city_state == city_state_0]
    temp_1 = casosGroupDf[casosGroupDf.city_state == city_state_1]

    for week in epi_week:
        temp2_0 = temp_0[temp_0.epidemiological_week == week]
        temp2_1 = temp_1[temp_1.epidemiological_week == week]
        if (temp2_0.empty) | (temp2_1.empty):
            new_confirmed_percent_0 = np.nan
            new_confirmed_percent_1 = np.nan
        elif temp2_0.empty:
            new_confirmed_percent_0 = 0
            new_confirmed_percent_1 = temp2_1.new_confirmed_percent.iloc[0]
        elif temp2_1.empty:
            new_confirmed_percent_1 = 0
            new_confirmed_percent_0 = temp2_0.new_confirmed_percent.iloc[0]
        
        else:
            new_confirmed_percent_0 = temp2_0.new_confirmed_percent.iloc[0]
            new_confirmed_percent_1 = temp2_1.new_confirmed_percent.iloc[0]
    
            new_confirmed_percent_avg = (new_confirmed_percent_0 + new_confirmed_percent_1)/2
            new_confirmed_percent_diff = new_confirmed_percent_0 * new_confirmed_percent_1
            new_confirmed_percent_avg_df.loc[week, str(edge)] = new_confirmed_percent_avg
            new_confirmed_percent_diff_df.loc[week, str(edge)] = new_confirmed_percent_diff

In [ ]:
new_confirmed_percent_avg_df.to_csv('new_confirmed_percent_avg_df_all.csv', sep=';')
new_confirmed_percent_diff_df.to_csv('new_confirmed_percent_diff_df_all.csv', sep=';')

In [ ]:
new_confirmed_percent_diff_df.tail()

In [26]:
for edge in G.edges():
    print(edge)
    city_state_a = edge[0]
    city_state_b = edge[1]
    temp_a = casosGroupDf[casosGroupDf.city_state == city_state_a]
    temp_b = casosGroupDf[casosGroupDf.city_state == city_state_b]

    for week in epi_week:
        temp2_a = temp_a[temp_a.epidemiological_week == week]
        temp2_b = temp_b[temp_b.epidemiological_week == week]
        
        try: confirmed_a = temp2_a.new_confirmed.iloc[0]
        except: confirmed_a = np.nan
        try: confirmed_b = temp2_b.new_confirmed.iloc[0]
        except: confirmed_b = np.nan
        try: confirmed_t1_a = temp2_a.new_confirmed_t1.iloc[0]
        except: confirmed_t1_a = np.nan
        try: confirmed_t1_b = temp2_b.new_confirmed_t1.iloc[0]
        except: confirmed_t1_b = np.nan
        try: confirmed_t2_a = temp2_a.new_confirmed_t2.iloc[0]
        except: confirmed_t2_a = np.nan
        try: confirmed_t2_b = temp2_b.new_confirmed_t2.iloc[0]
        except: confirmed_t2_b = np.nan
        #try: confirmed_t3_a = temp2_a.new_confirmed_t3.iloc[0]
        #except: confirmed_t3_a = np.nan
        #try: confirmed_t3_b = temp2_b.new_confirmed_t3.iloc[0]
        #except: confirmed_t3_b = np.nan
        #try: confirmed_t4_a = temp2_a.new_confirmed_t4.iloc[0]
        #except: confirmed_t4_a = np.nan
        #try: confirmed_t4_b = temp2_b.new_confirmed_t5.iloc[0]
        #except: confirmed_t4_b = np.nan
            
        if (pd.isna(confirmed_a)) | (pd.isna(confirmed_t1_a)) | (pd.isna(confirmed_t2_a)) | (pd.isna(confirmed_b)) | (pd.isna(confirmed_t1_b)) | (pd.isna(confirmed_t2_b)):
            corr = np.nan
        elif ((confirmed_a == confirmed_t1_a) & (confirmed_a == confirmed_t1_a)) | ((confirmed_b == confirmed_t1_b) & (confirmed_b == confirmed_t1_b)):
            corr = np.nan
        else:
        
            t0_a = [confirmed_t2_a,confirmed_t1_a,confirmed_a] 
            t0_b = [confirmed_t2_b,confirmed_t1_b,confirmed_b]
            try: corr, p = pearsonr(t0_a, t0_b)
            except: corr = np.nan
            
        new_confirmed_corr_df.loc[week, str(edge)] = corr



       

('ALTA FLORESTA DOESTE (RO)', 'NOVA BRASILANDIA DOESTE (RO)')
('ALTA FLORESTA DOESTE (RO)', 'ROLIM DE MOURA (RO)')
('ALTA FLORESTA DOESTE (RO)', 'SANTA LUZIA DOESTE (RO)')
('ALTA FLORESTA DOESTE (RO)', 'SAO MIGUEL DO GUAPORE (RO)')
('ALTA FLORESTA DOESTE (RO)', 'ALTO ALEGRE DOS PARECIS (RO)')
('ALTA FLORESTA DOESTE (RO)', 'NOVO HORIZONTE DO OESTE (RO)')
('ALTA FLORESTA DOESTE (RO)', 'SAO FRANCISCO DO GUAPORE (RO)')
('NOVA BRASILANDIA DOESTE (RO)', 'PRESIDENTE MEDICI (RO)')
('NOVA BRASILANDIA DOESTE (RO)', 'SAO MIGUEL DO GUAPORE (RO)')
('NOVA BRASILANDIA DOESTE (RO)', 'ALVORADA DOESTE (RO)')
('NOVA BRASILANDIA DOESTE (RO)', 'NOVO HORIZONTE DO OESTE (RO)')
('NOVA BRASILANDIA DOESTE (RO)', 'CASTANHEIRAS (RO)')
('ROLIM DE MOURA (RO)', 'CACOAL (RO)')
('ROLIM DE MOURA (RO)', 'PIMENTA BUENO (RO)')
('ROLIM DE MOURA (RO)', 'SANTA LUZIA DOESTE (RO)')
('ROLIM DE MOURA (RO)', 'NOVO HORIZONTE DO OESTE (RO)')
('ROLIM DE MOURA (RO)', 'CASTANHEIRAS (RO)')
('ROLIM DE MOURA (RO)', 'SAO FELIPE DOESTE (RO

('PORTO VELHO (RO)', 'FORTALEZA (CE)')
('PORTO VELHO (RO)', 'GUARULHOS (SP)')
('PORTO VELHO (RO)', 'MANAUS (AM)')
('PORTO VELHO (RO)', 'VARZEA GRANDE (MT)')
('PORTO VELHO (RO)', 'BOA VISTA (RR)')
('PORTO VELHO (RO)', 'SAO PAULO (SP)')
('PORTO VELHO (RO)', 'RECIFE (PE)')
('PORTO VELHO (RO)', 'RIO BRANCO (AC)')
('PORTO VELHO (RO)', 'RIO DE JANEIRO (RJ)')
('PORTO VELHO (RO)', 'TEFE (AM)')
('PORTO VELHO (RO)', 'COARI (AM)')
('PORTO VELHO (RO)', 'SAO CARLOS (SP)')
('PORTO VELHO (RO)', 'RIBEIRAO PRETO (SP)')
('PORTO VELHO (RO)', 'UBERLANDIA (MG)')
('PORTO VELHO (RO)', 'GOIANIA (GO)')
('PORTO VELHO (RO)', 'SANTAREM (PA)')
('CUJUBIM (RO)', 'CANDEIAS DO JAMARI (RO)')
('CUJUBIM (RO)', 'ITAPUA DO OESTE (RO)')
('HUMAITA (AM)', 'CANUTAMA (AM)')
('HUMAITA (AM)', 'MANICORE (AM)')
('HUMAITA (AM)', 'TAPAUA (AM)')
('MANICORE (AM)', 'BERURI (AM)')
('MANICORE (AM)', 'BORBA (AM)')
('MANICORE (AM)', 'NOVO ARIPUANA (AM)')
('MANICORE (AM)', 'TAPAUA (AM)')
('NOVO ARIPUANA (AM)', 'APUI (AM)')
('NOVO ARIPUANA (A

('APIACAS (MT)', 'COTRIGUACU (MT)')
('APIACAS (MT)', 'NOVA BANDEIRANTES (MT)')
('APIACAS (MT)', 'PARANAITA (MT)')
('APIACAS (MT)', 'NOVA MONTE VERDE (MT)')
('COTRIGUACU (MT)', 'ARIPUANA (MT)')
('COTRIGUACU (MT)', 'JURUENA (MT)')
('COTRIGUACU (MT)', 'NOVA BANDEIRANTES (MT)')
('BENJAMIN CONSTANT (AM)', 'TABATINGA (AM)')
('AUTAZES (AM)', 'CAREIRO (AM)')
('AUTAZES (AM)', 'CAREIRO DA VARZEA (AM)')
('AUTAZES (AM)', 'ITACOATIARA (AM)')
('AUTAZES (AM)', 'NOVA OLINDA DO NORTE (AM)')
('CAREIRO (AM)', 'CAREIRO DA VARZEA (AM)')
('CAREIRO (AM)', 'MANAQUIRI (AM)')
('CAREIRO DA VARZEA (AM)', 'IRANDUBA (AM)')
('CAREIRO DA VARZEA (AM)', 'ITACOATIARA (AM)')
('CAREIRO DA VARZEA (AM)', 'MANAQUIRI (AM)')
('CAREIRO DA VARZEA (AM)', 'MANAUS (AM)')
('ITACOATIARA (AM)', 'BOA VISTA DO RAMOS (AM)')
('ITACOATIARA (AM)', 'ITAPIRANGA (AM)')
('ITACOATIARA (AM)', 'MANAUS (AM)')
('ITACOATIARA (AM)', 'NOVA OLINDA DO NORTE (AM)')
('ITACOATIARA (AM)', 'RIO PRETO DA EVA (AM)')
('ITACOATIARA (AM)', 'SILVES (AM)')
('ITACOAT

('PONTA DE PEDRAS (PA)', 'ANAJAS (PA)')
('PONTA DE PEDRAS (PA)', 'BELEM (PA)')
('PONTA DE PEDRAS (PA)', 'CACHOEIRA DO ARARI (PA)')
('PONTA DE PEDRAS (PA)', 'SANTA CRUZ DO ARARI (PA)')
('ABEL FIGUEIREDO (PA)', 'BOM JESUS DO TOCANTINS (PA)')
('ABEL FIGUEIREDO (PA)', 'RONDON DO PARA (PA)')
('ABEL FIGUEIREDO (PA)', 'SAO PEDRO DA AGUA BRANCA (MA)')
('ABEL FIGUEIREDO (PA)', 'VILA NOVA DOS MARTIRIOS (MA)')
('BOM JESUS DO TOCANTINS (PA)', 'MARABA (PA)')
('BOM JESUS DO TOCANTINS (PA)', 'RONDON DO PARA (PA)')
('BOM JESUS DO TOCANTINS (PA)', 'SAO JOAO DO ARAGUAIA (PA)')
('BOM JESUS DO TOCANTINS (PA)', 'ESPERANTINA (TO)')
('BOM JESUS DO TOCANTINS (PA)', 'SAO PEDRO DA AGUA BRANCA (MA)')
('RONDON DO PARA (PA)', 'DOM ELISEU (PA)')
('RONDON DO PARA (PA)', 'GOIANESIA DO PARA (PA)')
('RONDON DO PARA (PA)', 'JACUNDA (PA)')
('RONDON DO PARA (PA)', 'MARABA (PA)')
('RONDON DO PARA (PA)', 'NOVA IPIXUNA (PA)')
('RONDON DO PARA (PA)', 'ACAILANDIA (MA)')
('RONDON DO PARA (PA)', 'CIDELANDIA (MA)')
('RONDON DO PA

('ALMEIRIM (PA)', 'MONTE ALEGRE (PA)')
('ALMEIRIM (PA)', 'OBIDOS (PA)')
('ALMEIRIM (PA)', 'PORTO DE MOZ (PA)')
('ALMEIRIM (PA)', 'PRAINHA (PA)')
('ALMEIRIM (PA)', 'LARANJAL DO JARI (AP)')
('ALMEIRIM (PA)', 'VITORIA DO JARI (AP)')
('ALMEIRIM (PA)', 'ALTAMIRA (PA)')
('ALMEIRIM (PA)', 'SANTAREM (PA)')
('CURUA (PA)', 'OBIDOS (PA)')
('CURUA (PA)', 'SANTAREM (PA)')
('MONTE ALEGRE (PA)', 'PRAINHA (PA)')
('MONTE ALEGRE (PA)', 'SANTAREM (PA)')
('MONTE ALEGRE (PA)', 'BREJO (MA)')
('MONTE ALEGRE (PA)', 'OBIDOS (PA)')
('OBIDOS (PA)', 'SANTAREM (PA)')
('OBIDOS (PA)', 'BREJO (MA)')
('SANTAREM (PA)', 'BELTERRA (PA)')
('SANTAREM (PA)', 'MOJUI DOS CAMPOS (PA)')
('SANTAREM (PA)', 'PRAINHA (PA)')
('SANTAREM (PA)', 'URUARA (PA)')
('SANTAREM (PA)', 'ALTAMIRA (PA)')
('SANTAREM (PA)', 'BRASILIA (DF)')
('SANTAREM (PA)', 'BREJO (MA)')
('SANTAREM (PA)', 'CAMPINAS (SP)')
('SANTAREM (PA)', 'CONFINS (MG)')
('SANTAREM (PA)', 'FORTALEZA (CE)')
('SANTAREM (PA)', 'GUARULHOS (SP)')
('SANTAREM (PA)', 'MARABA (PA)')
('SA

...

('ACORIZAL (MT)', 'JANGADA (MT)')
('ACORIZAL (MT)', 'ROSARIO OESTE (MT)')
('ACORIZAL (MT)', 'VARZEA GRANDE (MT)')
('CHAPADA DOS GUIMARAES (MT)', 'CAMPO VERDE (MT)')
('CHAPADA DOS GUIMARAES (MT)', 'CUIABA (MT)')
('CHAPADA DOS GUIMARAES (MT)', 'NOVA BRASILANDIA (MT)')
('CHAPADA DOS GUIMARAES (MT)', 'ROSARIO OESTE (MT)')
('CUIABA (MT)', 'CAMPO VERDE (MT)')
('CUIABA (MT)', 'NOSSA SENHORA DO LIVRAMENTO (MT)')
('CUIABA (MT)', 'VARZEA GRANDE (MT)')
('JANGADA (MT)', 'NOSSA SENHORA DO LIVRAMENTO (MT)')
('JANGADA (MT)', 'ROSARIO OESTE (MT)')
('JANGADA (MT)', 'VARZEA GRANDE (MT)')
('ROSARIO OESTE (MT)', 'ALTO PARAGUAI (MT)')
('ROSARIO OESTE (MT)', 'BARRA DO BUGRES (MT)')
('ROSARIO OESTE (MT)', 'NOBRES (MT)')
('ROSARIO OESTE (MT)', 'NOSSA SENHORA DO LIVRAMENTO (MT)')
('ROSARIO OESTE (MT)', 'NOVA BRASILANDIA (MT)')
('ROSARIO OESTE (MT)', 'PARANATINGA (MT)')
('ROSARIO OESTE (MT)', 'PLANALTO DA SERRA (MT)')
('ROSARIO OESTE (MT)', 'PORTO ESTRELA (MT)')
('ROSARIO OESTE (MT)', 'SANTA RITA DO TRIVELATO (

('JAURU (MT)', 'VALE DE SAO DOMINGOS (MT)')
('SAO JOSE DOS QUATRO MARCOS (MT)', 'FIGUEIROPOLIS DOESTE (MT)')
('SAO JOSE DOS QUATRO MARCOS (MT)', 'GLORIA DOESTE (MT)')
('SAO JOSE DOS QUATRO MARCOS (MT)', 'LAMBARI DOESTE (MT)')
('SAO JOSE DOS QUATRO MARCOS (MT)', 'MIRASSOL DOESTE (MT)')
('SAO JOSE DOS QUATRO MARCOS (MT)', 'PORTO ESPERIDIAO (MT)')
('SAO JOSE DOS QUATRO MARCOS (MT)', 'RIO BRANCO (MT)')
('RESERVA DO CABACAL (MT)', 'RIO BRANCO (MT)')
('RESERVA DO CABACAL (MT)', 'SALTO DO CEU (MT)')
('RESERVA DO CABACAL (MT)', 'TANGARA DA SERRA (MT)')
('RIO BRANCO (MT)', 'LAMBARI DOESTE (MT)')
('RIO BRANCO (MT)', 'SALTO DO CEU (MT)')
('SANTO AFONSO (MT)', 'TANGARA DA SERRA (MT)')
('SANTO AFONSO (MT)', 'NOVA MARILANDIA (MT)')
('NOVA MARILANDIA (MT)', 'TANGARA DA SERRA (MT)')
('ARIPUANA (MT)', 'JURUENA (MT)')
('ARIPUANA (MT)', 'TANGARA DA SERRA (MT)')
('JURUENA (MT)', 'CASTANHEIRA (MT)')
('NOSSA SENHORA DO LIVRAMENTO (MT)', 'PORTO ESTRELA (MT)')
('LAMBARI DOESTE (MT)', 'CURVELANDIA (MT)')
('LAM

('GOIANIA (GO)', 'TRINDADE (GO)')
('GOIANIA (GO)', 'RIO VERDE (GO)')
('GUAPO (GO)', 'CAMPESTRE DE GOIAS (GO)')
('GUAPO (GO)', 'CEZARINA (GO)')
('GUAPO (GO)', 'PALMEIRAS DE GOIAS (GO)')
('GUAPO (GO)', 'TRINDADE (GO)')
('GUAPO (GO)', 'VARJAO (GO)')
('TRINDADE (GO)', 'AVELINOPOLIS (GO)')
('TRINDADE (GO)', 'CAMPESTRE DE GOIAS (GO)')
('TRINDADE (GO)', 'CATURAI (GO)')
('TRINDADE (GO)', 'GOIANIRA (GO)')
('TRINDADE (GO)', 'SANTA BARBARA DE GOIAS (GO)')
('ABADIANIA (GO)', 'ALEXANIA (GO)')
('ABADIANIA (GO)', 'ANAPOLIS (GO)')
('ABADIANIA (GO)', 'CORUMBA DE GOIAS (GO)')
('ABADIANIA (GO)', 'GAMELEIRA DE GOIAS (GO)')
('ABADIANIA (GO)', 'PIRENOPOLIS (GO)')
('ABADIANIA (GO)', 'SILVANIA (GO)')
('ALEXANIA (GO)', 'CORUMBA DE GOIAS (GO)')
('ALEXANIA (GO)', 'LUZIANIA (GO)')
('ALEXANIA (GO)', 'SANTO ANTONIO DO DESCOBERTO (GO)')
('ALEXANIA (GO)', 'SILVANIA (GO)')
('ANAPOLIS (GO)', 'CAMPO LIMPO DE GOIAS (GO)')
('ANAPOLIS (GO)', 'GAMELEIRA DE GOIAS (GO)')
('ANAPOLIS (GO)', 'GOIANAPOLIS (GO)')
('ANAPOLIS (GO)',

('COLINAS DO SUL (GO)', 'CAMPINACU (GO)')
('NOVA ROMA (GO)', 'IACIARA (GO)')
('NOVA ROMA (GO)', 'TERESINA DE GOIAS (GO)')
('BURITINOPOLIS (GO)', 'SIMOLANDIA (GO)')
('AMARALINA (GO)', 'BONOPOLIS (GO)')
('AMARALINA (GO)', 'MUNDO NOVO (GO)')
('AMARALINA (GO)', 'MUTUNOPOLIS (GO)')
('AMARALINA (GO)', 'UIRAPURU (GO)')
('BONOPOLIS (GO)', 'MUNDO NOVO (GO)')
('MUNDO NOVO (GO)', 'UIRAPURU (GO)')
('MUTUNOPOLIS (GO)', 'ESTRELA DO NORTE (GO)')
('MUTUNOPOLIS (GO)', 'SANTA TEREZA DE GOIAS (GO)')
('UIRAPURU (GO)', 'CRIXAS (GO)')
('UIRAPURU (GO)', 'SANTA TEREZINHA DE GOIAS (GO)')
('AMERICANO DO BRASIL (GO)', 'ITABERAI (GO)')
('ITABERAI (GO)', 'ARACU (GO)')
('ITABERAI (GO)', 'AVELINOPOLIS (GO)')
('ITABERAI (GO)', 'GOIAS (GO)')
('ITABERAI (GO)', 'HEITORAI (GO)')
('ITABERAI (GO)', 'ITAGUARI (GO)')
('ITABERAI (GO)', 'ITAGUARU (GO)')
('ITABERAI (GO)', 'ITAUCU (GO)')
('ITABERAI (GO)', 'TAQUARAL DE GOIAS (GO)')
('ITABERAI (GO)', 'URUANA (GO)')
('AMORINOPOLIS (GO)', 'IPORA (GO)')
('AMORINOPOLIS (GO)', 'IVOLAND

('BRAZABRANTES (GO)', 'SANTO ANTONIO DE GOIAS (GO)')
('DAMOLANDIA (GO)', 'NOVA VENEZA (GO)')
('GOIANIRA (GO)', 'SANTO ANTONIO DE GOIAS (GO)')
('NOVA VENEZA (GO)', 'SANTO ANTONIO DE GOIAS (GO)')
('ITAPIRAPUA (GO)', 'GOIAS (GO)')
('ITAPIRAPUA (GO)', 'NOVO BRASIL (GO)')
('ITAPIRAPUA (GO)', 'SANTA FE DE GOIAS (GO)')
('BURITI DE GOIAS (GO)', 'GOIAS (GO)')
('BURITI DE GOIAS (GO)', 'NOVO BRASIL (GO)')
('GOIAS (GO)', 'GUARAITA (GO)')
('GOIAS (GO)', 'HEITORAI (GO)')
('GOIAS (GO)', 'ITAPURANGA (GO)')
('GOIAS (GO)', 'NOVO BRASIL (GO)')
('NOVO BRASIL (GO)', 'FAZENDA NOVA (GO)')
('CALDAS NOVAS (GO)', 'PIRES DO RIO (GO)')
('CALDAS NOVAS (GO)', 'SANTA CRUZ DE GOIAS (GO)')
('PIRES DO RIO (GO)', 'ORIZONA (GO)')
('PIRES DO RIO (GO)', 'PALMELO (GO)')
('PIRES DO RIO (GO)', 'SANTA CRUZ DE GOIAS (GO)')
('PIRES DO RIO (GO)', 'URUTAI (GO)')
('PIRES DO RIO (GO)', 'VIANOPOLIS (GO)')
('SANTA CRUZ DE GOIAS (GO)', 'PALMELO (GO)')
('CAMPESTRE DE GOIAS (GO)', 'PALMEIRAS DE GOIAS (GO)')
('PALMEIRAS DE GOIAS (GO)', 'C

In [27]:
new_confirmed_corr_df.to_csv('new_confirmed_corr_df_all.csv', sep=';')

In [ ]:
for edge in G.edges(data=True):
                if (temp[temp.city_state == edge[0]].empty) | (temp[temp.city_state == edge[1]].empty):
                    corr = np.nan
            else:
                confirmed_0 = temp[(temp.city_state == edge[0])].new_confirmed.iloc[0]
                confirmed_1 = temp[(temp.city_state == edge[1])].new_confirmed.iloc[0]
                confirmed_t1_0 = temp[(temp.city_state == edge[0])].new_confirmed_t1.iloc[0]
                confirmed_t1_1 = temp[(temp.city_state == edge[1])].new_confirmed_t1.iloc[0]
                confirmed_t2_0 = temp[(temp.city_state == edge[0])].new_confirmed_t2.iloc[0]
                confirmed_t2_1 = temp[(temp.city_state == edge[1])].new_confirmed_t2.iloc[0]
                confirmed_t3_0 = temp[(temp.city_state == edge[0])].new_confirmed_t3.iloc[0]
                confirmed_t3_1 = temp[(temp.city_state == edge[1])].new_confirmed_t3.iloc[0]
                confirmed_t4_0 = temp[(temp.city_state == edge[0])].new_confirmed_t4.iloc[0]
                confirmed_t4_1 = temp[(temp.city_state == edge[1])].new_confirmed_t5.iloc[0]
                confirmed_t5_0 = temp[(temp.city_state == edge[0])].new_confirmed_t5.iloc[0]
                confirmed_t5_1 = temp[(temp.city_state == edge[1])].new_confirmed_t5.iloc[0]
                confirmed_t6_0 = temp[(temp.city_state == edge[0])].new_confirmed_t6.iloc[0]
                confirmed_t6_1 = temp[(temp.city_state == edge[1])].new_confirmed_t6.iloc[0]
                
                t0_0 = [confirmed_t5_0,confirmed_t4_0,confirmed_t3_0,confirmed_t2_0,confirmed_t1_0,confirmed_0]
                #t1_0 = [confirmed_t6_0,confirmed_t5_0,confirmed_t4_0,confirmed_t3_0,confirmed_t2_0,confirmed_t1_0]
                
                t0_1 = [confirmed_t5_1,confirmed_t4_1,confirmed_t3_1,confirmed_t2_1,confirmed_t1_1,confirmed_1]
                #t1_1 = [confirmed_t6_1,confirmed_t5_1,confirmed_t4_1,confirmed_t3_1,confirmed_t2_1,confirmed_t1_1]
                
                try: corr_t0, p_t0 = pearsonr(t0_0, t0_1)
                except: corr_t0 = np.nan
                #try: corr_t1, p_t1 = pearsonr(t1_0, t1_1)
                #except: corr_t1 = np.nan
                #corr = abs(corr_t0) - abs(corr_t1)
                corr = corr_t0
            print(edge)
            

In [ ]:
import matplotlib as mp
import matplotlib.cm as cm

pearson = [x[2]['pearson'] for x in list(G.edges(data=True))]

minima = min(pearson)
maxima = max(pearson)

norm = mp.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=plt.cm.BrBG)

color_list = [mapper.to_rgba(v) for v in pearson]
alpha_list = [0.5 if x[2]['aereo'] == 'sim' else 0.5 for x in list(G.edges(data=True))]
linestyle_list = ['dotted' if x[2]['aereo'] == 'sim' else 'solid' for x in list(G.edges(data=True))]

capital = nx.get_node_attributes(G, 'capital')
region = nx.get_node_attributes(G, 'region')
node_size_list = [15 if x == 'sim' else 10 for x in list(capital.values())]
node_alpha_list = [0.6 if x == 'sim' else 0.4 for x in list(capital.values())]
node_color_list = 'gray'

labels = {}    
for node in G.nodes(data=True):
    if node[1]['capital'] == 'sim': 
        labels[node[0]] = node[0]

positions = nx.get_node_attributes(G, 'pos')
# Produce the curves
curves = curved_edges(G, positions)
lc = LineCollection(curves, color=color_list, alpha=alpha_list, linestyle=linestyle_list)

size = nx.get_node_attributes(G, 'population')
size_list = [v for v in size.values()]
size_norm = [float(i)/max(size_list) for i in size_list]

plt.figure(figsize=(20,20))
nx.draw_networkx_nodes(G, positions, node_size=node_size_list, alpha=node_alpha_list, node_color=node_color_list)
plt.gca().add_collection(lc)
plt.tick_params(axis='both',which='both',bottom=False,left=False,labelbottom=False,labelleft=False)
#nx.draw_networkx_labels(G, positions, labels, font_size=12, font_color="black")
plt.savefig('grafo_cidades_aeroportos_202040.png', transparent=False)
plt.show()

In [ ]:
minimum_spanning_tree = tree.mst.minimum_spanning_tree(G, weight="pearson", ignore_nan=True)
sum_ = sum(list(nx.get_edge_attributes( minimum_spanning_tree, 'pearson').values()))
print('Sum:', sum_)
#Sum: -517.7498406040883 202020
#Sum: -806.0413886847555 202022
#Sum: -1056.9493846100775 202025
#Sum: -1322.9335498411547 202031
#Sum: -1362.3329650691155 202033
#Sum: -1275.7802168555108 202035
#Sum: -1332.737400610826 202036
#Sum: -1232.9998790140364 202040
#Sum: -1225.2711675299283 202045
#Sum: -1177.700229849171 202050

In [ ]:
1 + np.exp(-1*sum_)
# 7.17627017190965e+224 202020
# inf 202022
# inf 202025
# inf 202031
# inf 202033
# 1.1934680253072109e+125 202036
# inf 202045


In [ ]:
sum_ = -806.0413886847555

In [ ]:
def infection_yes_no(input_network, casosDf, vis = False, title=""):

    print("Replicating infection.")

    network = input_network.copy()
    positions = nx.get_node_attributes(network , 'pos')
    curves = curved_edges(network , positions)

    # Set the default to susceptable
    sys.stdout.flush()
    for node in network.nodes():
        nx.set_node_attributes(network, values = {node:"s"}, name='status')
        nx.set_node_attributes(network, values = {node:"#A6A6A6"}, name='color')
        nx.set_node_attributes(network, values = {node: 0.4}, name='alpha')
    for edge in network.edges(data=True):
        nx.set_edge_attributes(network, {(edge[0],edge[1]): "#A6A6A6"}, 'color')
    if vis:
        pos = nx.get_node_attributes(network, 'pos')
        
    epi_weeks = sorted(list(set(casosDf.epidemiological_week.tolist())))
    
    for week in epi_weeks:
        inicial, infectado  = 0,0
        weekDf = casosDf[casosDf.epidemiological_week == week]
        cities = list(set(weekDf.city_state.tolist())) 
        for city in cities:
            nx.set_node_attributes(network, {city: 'i'}, 'status')
            nx.set_node_attributes(network, {city: (0.888292, 0.40830288, 0.36223756)}, 'color')
            nx.set_node_attributes(network, {city: 0.8}, 'alpha')
        for edge in network.edges(data=True):
            if network.nodes[edge[0]]['status'] == 'i' and network.nodes[edge[1]]['status'] == 'i':
                nx.set_edge_attributes(network, {(edge[0],edge[1]): (0.888292, 0.40830288, 0.36223756)}, 'color')
                
        for node in network.nodes():
            status = network.nodes[node]["status"]
            color = network.nodes[node]["color"]
            if status == "s":
                inicial += 1
            elif status == "i":         
                infectado += 1
        print("{0}, {1}, {2}".format(week, inicial/len(network.nodes()), infectado/len(network.nodes())))


        if inicial == 0:
            break

        if vis:
            visualize_curved_edges(network, title, pos, curves)
        
    print("\t----------\n\tS: {0}, I: {1}, {2}".format(inicial,infectado,infectado/len(network.nodes())))

    return {"Suscceptable":inicial,"Infected":infectado}

In [ ]:
def infection_cases(input_network, casosDf, DELAY=0, vis = False, 
              file_name = "sir.csv", title="",  RECALCULATE = True):
    
    print("Replicating infection.")

    network = input_network.copy()
    positions = nx.get_node_attributes(network , 'pos')
    curves = curved_edges(network , positions)

    # Set the default to susceptable
    sys.stdout.flush()
    for node in network.nodes():
        nx.set_node_attributes(network, values = {node:"s"}, name='status')
        nx.set_node_attributes(network, values = {node:"#A6A6A6"}, name='color')
        nx.set_node_attributes(network, values = {node: 0.4}, name='alpha')
    for edge in network.edges(data=True):
        nx.set_edge_attributes(network, {(edge[0],edge[1]): "#A6A6A6"}, 'color')
    
    if vis:
        pos = nx.get_node_attributes(network, 'pos')
    
    epi_weeks = sorted(list(set(casosDf.epidemiological_week.tolist())))
    palette = sns.color_palette("flare")
    # Iterate through the evolution of the disease.
    i = 0
    for week in epi_weeks:
        i = i + 1
        print(i)
        # Create variables to hold the outcomes as they happen
        S,queda,estab,aumento = 0,0,0,0
        
        casosWeekDf = casosDf[casosDf.epidemiological_week == week]
        cities = list(set(casosWeekDf.city_state.tolist()))
        
        for city in cities:
            new_per = casosWeekDf[casosWeekDf.city_state == city].new_deaths_percent.values[0]
            if new_per < -0.01:
                nx.set_node_attributes(network, {city: 'em queda'}, 'status')
                nx.set_node_attributes(network, {city: palette[4]}, 'color')
                nx.set_node_attributes(network, {city: 0.7}, 'alpha')

            elif new_per < 0.10:
                nx.set_node_attributes(network, {city: 'estabilidade'}, 'status')
                nx.set_node_attributes(network, {city: palette[2]}, 'color')
                nx.set_node_attributes(network, {city: 0.7}, 'alpha')

            else:
                nx.set_node_attributes(network, {city: 'aumento'}, 'status')
                nx.set_node_attributes(network, {city: palette[0]}, 'color')
                nx.set_node_attributes(network, {city: 0.8}, 'alpha')

            
        # Loop twice to prevent bias.
        for edge in network.edges(data=True):
            if (casosWeekDf[(casosWeekDf.city_state == edge[0])].empty) | (casosWeekDf[(casosWeekDf.city_state == edge[1])].empty):
                pass
            else:
                confirmed_0 = casosWeekDf[(casosWeekDf.city_state == edge[0])].new_confirmed.iloc[0]
                confirmed_1 = casosWeekDf[(casosWeekDf.city_state == edge[1])].new_confirmed.iloc[0]
                last_week_confirmed_0 = casosWeekDf[(casosWeekDf.city_state == edge[0])].last_week_new_confirmed.iloc[0]
                last_week_confirmed_1 = casosWeekDf[(casosWeekDf.city_state == edge[1])].last_week_new_confirmed.iloc[0]
                corr, p = pearsonr([last_week_confirmed_0,confirmed_0], [last_week_confirmed_1,confirmed_1])
                nx.set_edge_attributes(network, {(edge[0],edge[1]): corr}, 'pearson')
            
            if network.nodes[edge[0]]['status'] == 'i' and network.nodes[edge[1]]['status'] == 'i':
                nx.set_edge_attributes(network, {(edge[0],edge[1]): (0.888292, 0.40830288, 0.36223756)}, 'color')
        
        minimum_spanning_tree = tree.mst.minimum_spanning_tree(network, weight="pearson")
        sum_ = sum(list(nx.get_edge_attributes( minimum_spanning_tree, 'pearson').values()))
        print('sum:', sum_)
        
        for node in network.nodes():
            status = network.nodes[node]["status"]
            color = network.nodes[node]["color"]

            if status == "s": S += 1
            if status == "em queda": queda += 1
            if status == "estabilidade": estab += 1
            elif status == "aumento": aumento += 1


        print("Semana: {0},Suscetivel: {1},Em queda: {2},Estabilidade: {3},Aumento: {4}".format(week, S, queda, estab, aumento))

        if vis:
            visualize_curved_edges(network, title, pos, curves)
        
    print("\t----------\n\tS: {0}, I: {1}, R: {2}".format(S,queda,aumento))

    return {"Suscceptable":S,"Infected":queda, "Recovered":aumento}

In [ ]:
from scipy.stats import pearsonr
from networkx.algorithms import tree
#minimum_spanning_tree_G = tree.mst.minimum_spanning_tree(G, weight="distance")
#G = create_network('grafo_cidades_aeroportos.gexf')
#for edge in G.edges(data=True):
#    print(edge[0],edge[1])
temp = casosDf.epidemiological_week[0]
temp2 = casosDf[casosDf.epidemiological_week == temp]
len(temp2)

In [ ]:
def visualize_curved_edges(network, title,pos, curves):
    """
    Visualize the network given an array of posisitons.
    """
    print("-- Starting to Visualize --")

    colors = []
    alphas = []
    edge_colors = []
    for node in network.nodes():
        colors.append(network.nodes[node]["color"])
        alphas.append(network.nodes[node]["alpha"])
    for edge in network.edges(data=True):
        edge_colors.append(network[edge[0]][edge[1]]["color"])
   
    positions = nx.get_node_attributes(network, 'pos')
    lc = LineCollection(curves, color=edge_colors, alpha=0.1)
    capital = nx.get_node_attributes(network, 'capital')
    sizes = [100 if x == 'sim' else 15 for x in list(capital.values())]
    # Plot
    fig, ax = plt.subplots(figsize=(8,8))
    nx.draw_networkx_nodes(network, positions, node_size=sizes, alpha=alphas, node_color = colors, ax=ax)
    plt.gca().add_collection(lc)
    plt.tick_params(axis='both',which='both',bottom=False,left=False,labelbottom=False,labelleft=False)

    number_files = str(len(os.listdir()))
    while len(number_files) < 3:
        number_files = "0" + number_files

    plt.show()

In [ ]:
def visualize(network, title,pos):
    """
    Visualize the network given an array of posisitons.
    """
    print("-- Starting to Visualize --")
    
    colors = []
    i_edge_colors = []
    d_edge_colors = []
    default = []
    infected = []
    for node in network.nodes():
        colors.append(network.nodes[node]["color"])
    for i,j in network.edges():
        color = network.nodes[i]["color"]
        alpha = 0.75
        if color == (0.42355299, 0.16934709, 0.42581586) or color == (0.48942421, 0.72854938, 0.56751036):
            color = "#A6A6A6"
            default.append((i,j))
            d_edge_colors.append(color)
        else:
            color = "#A6A6A6"#29A229"
            infected.append((i,j))
            i_edge_colors.append(color)

    fig, ax = plt.subplots(figsize=(10,10))

    # Fist pass - Gray lines
    nx.draw_networkx_edges(network,pos,connectionstyle="arc3,rad=0.9",
                           edgelist=default,
            width=0.5,
            edge_color=d_edge_colors,
            alpha=0.5,
            arrows=False, ax=ax)
   
     #Second Pass - Colored lines
    nx.draw_networkx_edges(network,pos,connectionstyle="arc3,rad=0.9",
                           edgelist=infected,
            width=0.5,
            edge_color=i_edge_colors,
            alpha=0.75,
            arrows=False, ax=ax)
    
    positions = nx.get_node_attributes(G, 'pos')
    nx.draw_networkx_nodes(network,
            pos,
            linewidths=0.5,
            node_size = 15,
            alpha=0.5,               
            #with_labels=False,
            node_color = colors, ax=ax)
    
    # Adjust the plot limits
    cut = 1.05
    xmax = cut * max(xx for xx,yy in pos.values())
    xmin =  min(xx for xx,yy in pos.values())
    xmin = xmin - (cut * xmin)

    ymax = cut * max(yy for xx,yy in pos.values())
    ymin = (cut) * min(yy for xx,yy in pos.values())
    ymin = ymin - (cut * ymin)

    number_files = str(len(os.listdir()))
    while len(number_files) < 3:
        number_files = "0" + number_files

    plt.show()
    plt.close()

In [ ]:
try: 
  G = create_network('/content/gdrive/MyDrive/Graph Theory for Machine Learning/gcavali-repo-main/grafo_cidades_aeroportos.gexf')
except:
  G = create_network('grafo_cidades_aeroportos.gexf')
infection_yes_no(G, casosGroupDf, vis=True, title='title', )

In [ ]:
try: 
  G = create_network('/content/gdrive/MyDrive/Graph Theory for Machine Learning/gcavali-repo-main/grafo_cidades_aeroportos.gexf')
except:
  G = create_network('grafo_cidades_aeroportos.gexf')
infection_cases(G, casosGroupDf, vis=True, title='title', )

In [ ]:
casosGroupDf['state'] = casosGroupDf.city_state.str.get(-3)+casosGroupDf.city_state.str.get(-2)

In [ ]:
try: 
  G = create_network('/content/gdrive/MyDrive/Graph Theory for Machine Learning/gcavali-repo-main/grafo_cidades.gexf')
except:
  G = create_network('grafo_cidades.gexf')
G.remove_edges_from(nx.selfloop_edges(G))
selected_nodes = [n for n,v in G.nodes(data=True) if n[-3:-1] == 'SP'] 
G_sub = G.subgraph(selected_nodes)
infection_yes_no(G_sub, casosGroupDf[casosGroupDf.state == 'SP'], vis=True, title='title', )

In [ ]:
infection_cases(G_sub, casosGroupDf[casosGroupDf.state == 'SP'], vis=True,title='title', DELAY=0)